# Beating the NBA Bookmakers Odds

In [64]:
import pandas as pd
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [65]:
import warnings
warnings.filterwarnings('ignore')

## Loading the Data and Preprocessing

In [66]:
boxscores = pd.read_csv("data/games.csv", index_col=0)

display(boxscores.head())
display(boxscores.tail())

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
1,240.0,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
2,240.0,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,...,20.0,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False
3,240.0,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,...,28.6,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True
4,240.0,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,20.0,...,16.8,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False


,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
17767,240.0,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
17768,240.0,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,37.0,...,25.0,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True
17769,240.0,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,20.0,...,25.6,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False
17770,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True
17771,240.0,240.0,33.0,85.0,0.388,12.0,44.0,0.273,28.0,34.0,...,51.5,36.2,141.0,114.0,BOS,117,0,2020,2020-09-19,False


In [67]:
boxscore = boxscores.sort_values("date")
boxscores = boxscores.reset_index(drop=True)
del boxscores["mp.1"]
del boxscores["mp_opp.1"]
del boxscores["index_opp"]

In [68]:
display(boxscores.head())
display(boxscores.tail())

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,0.778,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
1,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,0.842,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
2,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,0.739,...,20.0,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False
3,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,0.895,...,28.6,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True
4,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,20.0,0.750,...,16.8,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False


,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
17767,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,0.750,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
17768,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,37.0,0.703,...,25.0,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True
17769,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,20.0,0.500,...,25.6,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False
17770,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,0.867,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True
17771,240.0,33.0,85.0,0.388,12.0,44.0,0.273,28.0,34.0,0.824,...,51.5,36.2,141.0,114.0,BOS,117,0,2020,2020-09-19,False


In [69]:
# Creating the Target Variable to Preiict the Winner of the Game
def add_target(group):
    group["target"] = group["won"].shift(-1)
    return group

boxscores = boxscores.groupby("team", group_keys=False).apply(add_target)

boxscores 

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,0.778,...,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True,True
1,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,0.842,...,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False,False
2,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,0.739,...,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False,True
3,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,0.895,...,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True,False
4,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,20.0,0.750,...,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,0.750,...,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True,True
17768,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,37.0,0.703,...,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True,NaN
17769,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,20.0,0.500,...,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False,NaN
17770,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,0.867,...,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True,NaN


In [71]:
boxscores["target"][pd.isnull(boxscores["target"])] = 2
boxscores["target"] = boxscores["target"].astype(int, errors="ignore")
boxscores 

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,0.778,...,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True,1
1,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,0.842,...,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False,0
2,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,0.739,...,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False,1
3,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,0.895,...,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True,0
4,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,20.0,0.750,...,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,0.750,...,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True,1
17768,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,37.0,0.703,...,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True,2
17769,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,20.0,0.500,...,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False,2
17770,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,0.867,...,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True,2


In [72]:
display(boxscores[boxscores["team"] == "NYK"].head)
display(boxscores[boxscores["team"] == "LAL"].head)
display(boxscores[boxscores["team"] == "BOS"].head)


<bound method NDFrame.head of           mp    fg   fga    fg%    3p   3pa    3p%    ft   fta    ft%  ...  \
114    240.0  36.0  86.0  0.419   3.0  18.0  0.167  16.0  20.0  0.800  ...   
121    240.0  41.0  88.0  0.466   3.0  18.0  0.167  25.0  26.0  0.962  ...   
206    240.0  39.0  91.0  0.429   8.0  28.0  0.286  22.0  32.0  0.688  ...   
220    240.0  39.0  87.0  0.448  12.0  29.0  0.414  10.0  12.0  0.833  ...   
228    240.0  36.0  82.0  0.439  13.0  30.0  0.433   7.0  11.0  0.636  ...   
...      ...   ...   ...    ...   ...   ...    ...   ...   ...    ...  ...   
17636  240.0  36.0  88.0  0.409   5.0  22.0  0.227  18.0  20.0  0.900  ...   
17639  240.0  40.0  82.0  0.488  13.0  32.0  0.406  18.0  24.0  0.750  ...   
17701  240.0  40.0  88.0  0.455   7.0  21.0  0.333  19.0  24.0  0.792  ...   
17745  240.0  37.0  82.0  0.451   9.0  28.0  0.321  14.0  15.0  0.933  ...   
17765  240.0  35.0  97.0  0.361   8.0  29.0  0.276  17.0  21.0  0.810  ...   

       usg%_max_opp  ortg_max_opp

<bound method NDFrame.head of           mp    fg   fga    fg%    3p   3pa    3p%    ft   fta    ft%  ...  \
37     240.0  34.0  82.0  0.415  11.0  40.0  0.275  17.0  23.0  0.739  ...   
44     240.0  39.0  78.0  0.500  10.0  30.0  0.333  28.0  43.0  0.651  ...   
55     240.0  32.0  90.0  0.356   4.0  19.0  0.211  13.0  17.0  0.765  ...   
58     240.0  43.0  91.0  0.473   8.0  26.0  0.308  16.0  23.0  0.696  ...   
74     240.0  43.0  90.0  0.478  10.0  30.0  0.333  14.0  17.0  0.824  ...   
...      ...   ...   ...    ...   ...   ...    ...   ...   ...    ...  ...   
17547  240.0  31.0  86.0  0.360   6.0  23.0  0.261  22.0  26.0  0.846  ...   
17585  240.0  43.0  86.0  0.500  14.0  29.0  0.483  23.0  28.0  0.821  ...   
17586  240.0  36.0  82.0  0.439   7.0  23.0  0.304  17.0  23.0  0.739  ...   
17592  240.0  31.0  83.0  0.373   3.0  21.0  0.143  12.0  16.0  0.750  ...   
17615  240.0  47.0  85.0  0.553  19.0  41.0  0.463  26.0  30.0  0.867  ...   

       usg%_max_opp  ortg_max_opp

<bound method NDFrame.head of           mp    fg   fga    fg%    3p   3pa    3p%    ft   fta    ft%  ...  \
43     240.0  42.0  91.0  0.462  17.0  42.0  0.405  17.0  25.0  0.680  ...   
135    240.0  42.0  86.0  0.488  16.0  32.0  0.500  17.0  23.0  0.739  ...   
141    240.0  45.0  83.0  0.542  12.0  33.0  0.364  26.0  28.0  0.929  ...   
176    240.0  30.0  81.0  0.370   7.0  25.0  0.280  26.0  36.0  0.722  ...   
181    240.0  42.0  89.0  0.472  12.0  31.0  0.387  12.0  14.0  0.857  ...   
...      ...   ...   ...    ...   ...   ...    ...   ...   ...    ...  ...   
17662  240.0  36.0  89.0  0.404  12.0  34.0  0.353  15.0  17.0  0.882  ...   
17677  240.0  39.0  93.0  0.419  12.0  39.0  0.308  15.0  19.0  0.789  ...   
17690  240.0  28.0  88.0  0.318   5.0  28.0  0.179  11.0  12.0  0.917  ...   
17734  265.0  39.0  97.0  0.402  14.0  37.0  0.378  24.0  31.0  0.774  ...   
17770  240.0  41.0  85.0  0.482   9.0  26.0  0.346  26.0  30.0  0.867  ...   

       usg%_max_opp  ortg_max_opp

In [73]:
display(boxscores["won"].value_counts())
# display(boxscores["target"].value_counts())

won
True     8886
False    8886
Name: count, dtype: int64

In [74]:
# Determine the columns that can be trained on
# We will ignore the columns that are not available at the start of the game (that are null)

nulls = pd.isnull(boxscores).sum()
nulls = nulls[nulls > 0]
non_null_columns = boxscores.columns[~boxscores.columns.isin(nulls.index)]

non_null_columns


Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'usg%_max_opp', 'ortg_max_opp', 'drtg_max_opp', 'team_opp', 'total_opp',
       'home_opp', 'season', 'date', 'won', 'target'],
      dtype='object', length=142)

In [75]:
boxscores = boxscores[non_null_columns].copy()
display(boxscores.head())
display(boxscores.tail())

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,0.778,...,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True,1
1,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,0.842,...,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False,0
2,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,0.739,...,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False,1
3,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,0.895,...,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True,0
4,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,20.0,0.750,...,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False,1


,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
17767,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,0.750,...,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True,1
17768,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,37.0,0.703,...,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True,2
17769,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,20.0,0.500,...,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False,2
17770,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,0.867,...,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True,2
17771,240.0,33.0,85.0,0.388,12.0,44.0,0.273,28.0,34.0,0.824,...,36.2,141.0,114.0,BOS,117,0,2020,2020-09-19,False,2


In [76]:
# Min-Max Scaler:
# We will use the Min-Max Scaler to all statistical features in the dataset to get them wihtin a range of 0 to 1
# We do this to normalize and reduces the variance of the data while preserving the relationships between the different 
# features as well as the relative order, distribution, and distnace of the datapoints

scaler = MinMaxScaler()

non_stat_columns = ["season", "date", "won", "target", "team", "team_opp"]
stat_columns = boxscores.columns[~boxscores.columns.isin(non_stat_columns)]

boxscores[stat_columns] = scaler.fit_transform(boxscores[stat_columns])

display(boxscores.head())
display(boxscores.tail())

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,0.0,0.454545,0.308824,0.507177,0.206897,0.242424,0.356295,0.302326,0.269841,0.740957,...,0.088575,0.421801,0.400000,DAL,0.298077,1.0,2016,2015-12-09,True,1
1,0.0,0.386364,0.588235,0.217703,0.241379,0.409091,0.268409,0.348837,0.285714,0.815636,...,0.134788,0.298578,0.400000,ATL,0.326923,0.0,2016,2015-12-09,False,0
2,0.0,0.409091,0.367647,0.397129,0.275862,0.227273,0.500000,0.372093,0.349206,0.695449,...,0.112965,0.279621,0.458824,SAS,0.413462,1.0,2018,2017-10-18,False,1
3,0.0,0.500000,0.426471,0.459330,0.275862,0.257576,0.452494,0.372093,0.285714,0.877480,...,0.112965,0.232227,0.482353,MIN,0.336538,0.0,2018,2017-10-18,True,0
4,0.0,0.181818,0.382353,0.107656,0.206897,0.333333,0.274347,0.325581,0.301587,0.708285,...,0.112965,0.322275,0.152941,MEM,0.269231,1.0,2021,2021-04-30,False,1


,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
17767,0.0,0.363636,0.308824,0.389952,0.379310,0.333333,0.502375,0.604651,0.555556,0.708285,...,0.148909,0.336493,0.482353,OKC,0.269231,0.0,2019,2018-10-19,True,1
17768,0.0,0.409091,0.205882,0.552632,0.448276,0.318182,0.617577,0.581395,0.571429,0.653442,...,0.101412,0.236967,0.611765,ORL,0.423077,1.0,2017,2016-12-14,True,2
17769,0.0,0.522727,0.426471,0.485646,0.482759,0.439394,0.503563,0.209302,0.301587,0.416569,...,0.100128,0.407583,0.576471,LAC,0.471154,0.0,2017,2016-12-14,False,2
17770,0.0,0.500000,0.367647,0.509569,0.310345,0.333333,0.410926,0.581395,0.460317,0.844807,...,0.064185,0.289100,0.576471,MIA,0.403846,1.0,2020,2020-09-19,True,2
17771,0.0,0.318182,0.367647,0.284689,0.413793,0.606061,0.324228,0.627907,0.523810,0.794632,...,0.181001,0.246445,0.435294,BOS,0.509615,0.0,2020,2020-09-19,False,2


In [77]:
boxscores.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

home
0.0    0.428314
1.0    0.571686
dtype: float64

In [78]:
# In our model we will be training on data based on an n-game lookback window (rolling average)
# Our objective in doing so is to better model and take into account the team's recent form and performance 
# when predicting the outcome of a game

# We will create a function that will create a rolling average of the data for each team for each game
bs_rolling = boxscores[list(stat_columns) + ["won", "team", "season"]]

def team_averages(group):
    group[stat_columns] = group[stat_columns].rolling(10).mean()
    return group

# We will then apply this function to each team in the dataset
bs_rolling = bs_rolling.groupby(["team", "season"], group_keys=False).apply(team_averages)


In [79]:
bs_rolling

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,won,team,season
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,ATL,2016
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,DAL,2016
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,MIN,2018
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,SAS,2018
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,ORL,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.025,0.434091,0.413235,0.392344,0.389655,0.378788,0.465083,0.553488,0.461905,0.803734,...,0.0699,0.282914,0.215019,0.408057,0.480000,0.525000,0.4,True,LAC,2019
17768,0.000,0.465909,0.295588,0.533014,0.382759,0.346970,0.496200,0.460465,0.412698,0.753092,...,0.0520,0.334696,0.122593,0.329858,0.544706,0.365385,0.4,True,LAC,2017
17769,0.050,0.495455,0.419118,0.455742,0.351724,0.343939,0.446793,0.316279,0.304762,0.698250,...,0.0821,0.307547,0.127599,0.409005,0.465882,0.398077,0.5,False,ORL,2017
17770,0.025,0.502273,0.427941,0.460526,0.427586,0.463636,0.436698,0.393023,0.322222,0.800000,...,0.1086,0.322117,0.150578,0.440284,0.528235,0.420192,0.3,True,BOS,2020


In [80]:
bs_rolling.columns

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'blk%_max_opp', 'tov%_max_opp', 'usg%_max_opp', 'ortg_max_opp',
       'drtg_max_opp', 'total_opp', 'home_opp', 'won', 'team', 'season'],
      dtype='object', length=139)

In [81]:
rolling_cols = [f"{col}_10" for col in bs_rolling.columns]
bs_rolling.columns = rolling_cols
boxscores = pd.concat([boxscores, bs_rolling], axis=1)

boxscores = boxscores.dropna()
boxscores = boxscores.reset_index(drop=True)

boxscores

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp_10,tov%_max_opp_10,usg%_max_opp_10,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,won_10,team_10,season_10
0,0.0,0.431818,0.264706,0.521531,0.275862,0.272727,0.432304,0.325581,0.301587,0.708285,...,0.0735,0.389937,0.243261,0.434123,0.485882,0.375962,0.6,True,MIL,2018
1,0.0,0.272727,0.235294,0.332536,0.275862,0.363636,0.339667,0.372093,0.349206,0.695449,...,0.0535,0.294444,0.165854,0.337441,0.468235,0.358654,0.5,False,SAS,2018
2,0.0,0.681818,0.529412,0.576555,0.206897,0.333333,0.274347,0.279070,0.206349,0.917153,...,0.0527,0.299790,0.160847,0.342180,0.490588,0.360577,0.5,True,SAS,2018
3,0.0,0.568182,0.544118,0.442584,0.724138,0.606061,0.566508,0.209302,0.174603,0.805134,...,0.0639,0.400105,0.175225,0.462085,0.560000,0.337500,0.6,False,MIA,2022
4,0.0,0.477273,0.455882,0.409091,0.517241,0.590909,0.414489,0.255814,0.222222,0.766628,...,0.1438,0.320335,0.151091,0.432701,0.512941,0.372115,0.7,True,GSW,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15877,0.0,0.363636,0.308824,0.389952,0.379310,0.333333,0.502375,0.604651,0.555556,0.708285,...,0.0699,0.282914,0.215019,0.408057,0.480000,0.525000,0.4,True,LAC,2019
15878,0.0,0.409091,0.205882,0.552632,0.448276,0.318182,0.617577,0.581395,0.571429,0.653442,...,0.0520,0.334696,0.122593,0.329858,0.544706,0.365385,0.4,True,LAC,2017
15879,0.0,0.522727,0.426471,0.485646,0.482759,0.439394,0.503563,0.209302,0.301587,0.416569,...,0.0821,0.307547,0.127599,0.409005,0.465882,0.398077,0.5,False,ORL,2017
15880,0.0,0.500000,0.367647,0.509569,0.310345,0.333333,0.410926,0.581395,0.460317,0.844807,...,0.1086,0.322117,0.150578,0.440284,0.528235,0.420192,0.3,True,BOS,2020


In [82]:
# We are now shifting columns to determine the next opponent and game date for each team per instance

def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

boxscores["home_next"] = add_col(boxscores, "home")
boxscores["team_opp_next"] = add_col(boxscores, "team_opp")
boxscores["date_next"] = add_col(boxscores, "date")

boxscores

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,won_10,team_10,season_10,home_next,team_opp_next,date_next
0,0.0,0.431818,0.264706,0.521531,0.275862,0.272727,0.432304,0.325581,0.301587,0.708285,...,0.434123,0.485882,0.375962,0.6,True,MIL,2018,1.0,IND,2018-01-03
1,0.0,0.272727,0.235294,0.332536,0.275862,0.363636,0.339667,0.372093,0.349206,0.695449,...,0.337441,0.468235,0.358654,0.5,False,SAS,2018,1.0,MIN,2018-03-17
2,0.0,0.681818,0.529412,0.576555,0.206897,0.333333,0.274347,0.279070,0.206349,0.917153,...,0.342180,0.490588,0.360577,0.5,True,SAS,2018,0.0,NYK,2021-05-13
3,0.0,0.568182,0.544118,0.442584,0.724138,0.606061,0.566508,0.209302,0.174603,0.805134,...,0.462085,0.560000,0.337500,0.6,False,MIA,2022,1.0,ATL,2022-04-19
4,0.0,0.477273,0.455882,0.409091,0.517241,0.590909,0.414489,0.255814,0.222222,0.766628,...,0.432701,0.512941,0.372115,0.7,True,GSW,2022,1.0,HOU,2016-04-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15877,0.0,0.363636,0.308824,0.389952,0.379310,0.333333,0.502375,0.604651,0.555556,0.708285,...,0.408057,0.480000,0.525000,0.4,True,LAC,2019,0.0,ORL,2016-12-14
15878,0.0,0.409091,0.205882,0.552632,0.448276,0.318182,0.617577,0.581395,0.571429,0.653442,...,0.329858,0.544706,0.365385,0.4,True,LAC,2017,NaN,None,None
15879,0.0,0.522727,0.426471,0.485646,0.482759,0.439394,0.503563,0.209302,0.301587,0.416569,...,0.409005,0.465882,0.398077,0.5,False,ORL,2017,NaN,None,None
15880,0.0,0.500000,0.367647,0.509569,0.310345,0.333333,0.410926,0.581395,0.460317,0.844807,...,0.440284,0.528235,0.420192,0.3,True,BOS,2020,NaN,None,None


In [83]:
# We are creating a matchup dataframe by merging the home and opponenet team data for each game

matchup = boxscores.merge(boxscores[rolling_cols + ["team_opp_next", "date_next", "team"]], 
                          left_on=["team", "date_next"], right_on=["team_opp_next", "date_next"])

matchup

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp_10_y,ortg_max_opp_10_y,drtg_max_opp_10_y,total_opp_10_y,home_opp_10_y,won_10_y,team_10_y,season_10_y,team_opp_next_y,team_y
0,0.00,0.477273,0.426471,0.430622,0.379310,0.409091,0.421615,0.069767,0.079365,0.611435,...,0.176380,0.632701,0.523529,0.363462,0.4,False,DET,2016,IND,DET
1,0.00,0.272727,0.308824,0.272727,0.448276,0.439394,0.467933,0.302326,0.365079,0.513419,...,0.149294,0.457820,0.425882,0.384615,0.5,False,IND,2018,DET,IND
2,0.00,0.545455,0.338235,0.595694,0.413793,0.348485,0.527316,0.348837,0.380952,0.579930,...,0.121566,0.390521,0.441176,0.381731,0.5,False,MIL,2018,DET,MIL
3,0.00,0.409091,0.308824,0.449761,0.482759,0.530303,0.426366,0.372093,0.333333,0.735123,...,0.170218,0.408531,0.502353,0.341346,0.8,True,GSW,2022,DAL,GSW
4,0.00,0.477273,0.558824,0.332536,0.379310,0.500000,0.352732,0.093023,0.095238,0.666278,...,0.138126,0.366825,0.371765,0.375000,0.7,True,BRK,2016,PHI,BRK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15533,0.25,0.522727,0.544118,0.392344,0.275862,0.378788,0.327791,0.465116,0.476190,0.623104,...,0.266496,0.357346,0.450588,0.454808,0.3,False,NYK,2017,MIL,NYK
15534,0.25,0.409091,0.485294,0.308612,0.241379,0.272727,0.377672,0.581395,0.507937,0.752625,...,0.183569,0.577725,0.535294,0.490385,0.4,False,DET,2020,PHO,DET
15535,0.00,0.454545,0.250000,0.566986,0.310345,0.272727,0.485748,0.558140,0.412698,0.913652,...,0.129782,0.418483,0.464706,0.380769,0.3,True,LAC,2016,MEM,LAC
15536,0.00,0.568182,0.294118,0.672249,0.482759,0.469697,0.475059,0.372093,0.269841,0.934656,...,0.145058,0.338863,0.482353,0.423077,0.6,False,OKC,2018,LAC,OKC


In [84]:
matchup[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]]

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
0,IND,DET,DET,IND,2017-11-08
1,DET,IND,IND,DET,2017-11-08
2,DET,MIL,MIL,DET,2017-12-06
3,DAL,GSW,GSW,DAL,2022-01-05
4,PHI,BRK,BRK,PHI,2016-03-15
...,...,...,...,...,...
15533,MIL,NYK,NYK,MIL,2018-12-25
15534,PHO,DET,DET,PHO,2022-01-16
15535,MEM,LAC,LAC,MEM,2021-02-26
15536,LAC,OKC,OKC,LAC,2018-10-19


In [85]:
non_stat_columns = list(matchup.columns[matchup.dtypes == "object"]) + non_stat_columns

non_stat_columns

['team_x',
 'team_opp',
 'date',
 'team_10_x',
 'team_opp_next_x',
 'date_next',
 'team_10_y',
 'team_opp_next_y',
 'team_y',
 'season',
 'date',
 'won',
 'target',
 'team',
 'team_opp']

In [86]:
# We are now splitting our matchup data into the training and target matrices
# We will be removing the non-statistical columns from the training matrix and using this as X
# We will be using the target column as y

X_columns = matchup.columns[~matchup.columns.isin(non_stat_columns)]
X = matchup[X_columns]
y = matchup["target"]

In [87]:
X

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,stl%_max_opp_10_y,blk%_max_opp_10_y,tov%_max_opp_10_y,usg%_max_opp_10_y,ortg_max_opp_10_y,drtg_max_opp_10_y,total_opp_10_y,home_opp_10_y,won_10_y,season_10_y
0,0.00,0.477273,0.426471,0.430622,0.379310,0.409091,0.421615,0.069767,0.079365,0.611435,...,0.0658,0.0801,0.314256,0.176380,0.632701,0.523529,0.363462,0.4,False,2016
1,0.00,0.272727,0.308824,0.272727,0.448276,0.439394,0.467933,0.302326,0.365079,0.513419,...,0.0591,0.1264,0.406709,0.149294,0.457820,0.425882,0.384615,0.5,False,2018
2,0.00,0.545455,0.338235,0.595694,0.413793,0.348485,0.527316,0.348837,0.380952,0.579930,...,0.0435,0.0727,0.331237,0.121566,0.390521,0.441176,0.381731,0.5,False,2018
3,0.00,0.409091,0.308824,0.449761,0.482759,0.530303,0.426366,0.372093,0.333333,0.735123,...,0.0510,0.0871,0.246436,0.170218,0.408531,0.502353,0.341346,0.8,True,2022
4,0.00,0.477273,0.558824,0.332536,0.379310,0.500000,0.352732,0.093023,0.095238,0.666278,...,0.0828,0.0943,0.353145,0.138126,0.366825,0.371765,0.375000,0.7,True,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15533,0.25,0.522727,0.544118,0.392344,0.275862,0.378788,0.327791,0.465116,0.476190,0.623104,...,0.1096,0.1056,0.346541,0.266496,0.357346,0.450588,0.454808,0.3,False,2017
15534,0.25,0.409091,0.485294,0.308612,0.241379,0.272727,0.377672,0.581395,0.507937,0.752625,...,0.0586,0.0920,0.326310,0.183569,0.577725,0.535294,0.490385,0.4,False,2020
15535,0.00,0.454545,0.250000,0.566986,0.310345,0.272727,0.485748,0.558140,0.412698,0.913652,...,0.0633,0.0403,0.589099,0.129782,0.418483,0.464706,0.380769,0.3,True,2016
15536,0.00,0.568182,0.294118,0.672249,0.482759,0.469697,0.475059,0.372093,0.269841,0.934656,...,0.0593,0.0923,0.422222,0.145058,0.338863,0.482353,0.423077,0.6,False,2018


In [88]:
y

0        1
1        1
2        1
3        0
4        0
        ..
15533    1
15534    1
15535    0
15536    1
15537    1
Name: target, Length: 15538, dtype: int64

In [89]:
y.value_counts()

target
1    7772
0    7766
Name: count, dtype: int64

In [90]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Building the RNN Model

In [91]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [92]:
%pip install keras

Note: you may need to restart the kernel to use updated packages.


In [93]:
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import Embedding
from keras.layers import GRU

In [94]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, TimeDistributed, Dense
# Below is the Vanilla Recurrent Neural Network Model
# We will be using the Keras API to build our model
# We will be using the Embedding layer to create a dense vector representation of each team's data
# We will be using the GRU layer to create a recurrent neural network that will be able to take into account the
# sequential nature of the data
# We will be using the TimeDistributed layer to apply a dense layer to each time step of the GRU layer
# We will be using the binary crossentropy loss function and the adam optimizer


def create_rnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Embedding(input_dim=input_shape[0], output_dim=32, input_length=input_shape[1]))
    model.add(GRU(64, return_sequences=True))
    model.add(GRU(64, return_sequences=True))  # Additional hidden GRU layer
    model.add(GRU(64, return_sequences=True))  # Additional hidden GRU layer
    model.add(TimeDistributed(Dense(32, activation='relu')))
    model.add(GRU(32))
    model.add(Dense(num_classes, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model




In [95]:
rnn_model = create_rnn_model((X_train.shape[0], X_train.shape[1]), 1)
print(rnn_model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 413, 32)           397760    
                                                                 
 gru_4 (GRU)                 (None, 413, 64)           18816     
                                                                 
 gru_5 (GRU)                 (None, 413, 64)           24960     
                                                                 
 gru_6 (GRU)                 (None, 413, 64)           24960     
                                                                 
 time_distributed_1 (TimeDi  (None, 413, 32)           2080      
 stributed)                                                      
                                                                 
 gru_7 (GRU)                 (None, 32)                6336      
                                                      

In [96]:
# Convert your data to a type that TensorFlow supports
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')

In [97]:
rnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
350/350 [==============================] - 159s 433ms/step - loss: 0.6934 - accuracy: 0.4946 - val_loss: 0.6930 - val_accuracy: 0.5173
Epoch 2/10
350/350 [==============================] - 143s 408ms/step - loss: 0.6933 - accuracy: 0.4929 - val_loss: 0.6936 - val_accuracy: 0.4827
Epoch 3/10
350/350 [==============================] - 155s 443ms/step - loss: 0.6933 - accuracy: 0.5034 - val_loss: 0.6935 - val_accuracy: 0.4827
Epoch 4/10
350/350 [==============================] - 146s 417ms/step - loss: 0.6932 - accuracy: 0.5050 - val_loss: 0.6929 - val_accuracy: 0.5173
Epoch 5/10
350/350 [==============================] - 143s 409ms/step - loss: 0.6932 - accuracy: 0.5051 - val_loss: 0.6938 - val_accuracy: 0.4827
Epoch 6/10
350/350 [==============================] - 137s 390ms/step - loss: 0.6932 - accuracy: 0.5017 - val_loss: 0.6935 - val_accuracy: 0.4827
Epoch 7/10
350/350 [==============================] - 137s 393ms/step - loss: 0.6932 - accuracy: 0.5027 - val_loss: 0.6932 -

In [98]:
from sklearn.metrics import classification_report
import numpy as np

# Convert your test data to a type that TensorFlow supports
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

# Make predictions using the rnn_model
y_pred = rnn_model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print("Accuracy:", accuracy)

print(classification_report(y_test, y_pred_classes))



98/98 [==============================] - 10s 87ms/step
Accuracy: 0.5019305019305019
              precision    recall  f1-score   support

         0.0       0.50      1.00      0.67      1560
         1.0       0.00      0.00      0.00      1548

    accuracy                           0.50      3108
   macro avg       0.25      0.50      0.33      3108
weighted avg       0.25      0.50      0.34      3108



# Load in and Simulate Model with Betting Odds

In [148]:
bets = pd.read_csv("data/games.csv", index_col=0)
bets = bets[bets['season'] == 2020]
bets = bets.sort_values("date")
bets = bets[(bets['date'] >= '2019-10-01') & (bets['date'] <= '2020-01-26')]
bets

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
976,240.0,240.0,37.0,85.0,0.435,13.0,33.0,0.394,15.0,21.0,...,22.2,42.4,203.0,116.0,LAC,112,1,2020,2019-10-22,False
977,240.0,240.0,42.0,81.0,0.519,11.0,31.0,0.355,17.0,24.0,...,25.0,35.8,300.0,125.0,LAL,102,0,2020,2019-10-22,True
7883,265.0,265.0,42.0,103.0,0.408,14.0,40.0,0.350,32.0,38.0,...,25.0,37.7,146.0,124.0,NOP,122,0,2020,2019-10-22,True
7882,265.0,265.0,43.0,102.0,0.422,19.0,45.0,0.422,17.0,20.0,...,22.2,35.7,158.0,114.0,TOR,130,1,2020,2019-10-22,False
16465,240.0,240.0,49.0,98.0,0.500,11.0,33.0,0.333,15.0,18.0,...,53.2,36.6,144.0,121.0,SAC,95,0,2020,2019-10-23,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14969,240.0,240.0,54.0,93.0,0.581,15.0,35.0,0.429,29.0,34.0,...,17.9,35.0,181.0,148.0,WAS,133,0,2020,2020-01-26,True
14968,240.0,240.0,42.0,88.0,0.477,11.0,33.0,0.333,38.0,42.0,...,38.1,36.4,168.0,131.0,ATL,152,1,2020,2020-01-26,False
14267,240.0,240.0,44.0,92.0,0.478,16.0,36.0,0.444,19.0,29.0,...,50.0,33.8,160.0,127.0,BOS,108,0,2020,2020-01-26,True
14266,240.0,240.0,41.0,89.0,0.461,8.0,33.0,0.242,18.0,22.0,...,39.3,33.7,173.0,111.0,NOP,123,1,2020,2020-01-26,False


In [154]:
display(bets[bets['date'] == '2019-10-22'])
display(bets[bets['date'] == '2019-10-22']['team'])

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
976,240.0,240.0,37.0,85.0,0.435,13.0,33.0,0.394,15.0,21.0,...,22.2,42.4,203.0,116.0,LAC,112,1,2020,2019-10-22,False
977,240.0,240.0,42.0,81.0,0.519,11.0,31.0,0.355,17.0,24.0,...,25.0,35.8,300.0,125.0,LAL,102,0,2020,2019-10-22,True
7883,265.0,265.0,42.0,103.0,0.408,14.0,40.0,0.350,32.0,38.0,...,25.0,37.7,146.0,124.0,NOP,122,0,2020,2019-10-22,True
7882,265.0,265.0,43.0,102.0,0.422,19.0,45.0,0.422,17.0,20.0,...,22.2,35.7,158.0,114.0,TOR,130,1,2020,2019-10-22,False


976     LAL
977     LAC
7883    TOR
7882    NOP
Name: team, dtype: object

In [212]:
odds = pd.read_csv("data/nbaodds2019-20.csv", index_col=0)
odds

,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
Date,,,,,,,,,,,,
1022,501,V,NewOrleans,30,31,25,31,122,231.5,229.5,230,113
1022,502,H,Toronto,27,29,32,29,130,6.5,6.5,-280,6
1022,503,V,LALakers,25,29,31,17,102,227,3.5,-180,5
1022,504,H,LAClippers,22,40,23,27,112,1.5,224,150,110.5
1023,505,V,Detroit,27,27,29,36,119,210,211,240,104
...,...,...,...,...,...,...,...,...,...,...,...,...
126,522,H,NewYork,30,25,30,25,110,221.5,2.5,-140,0.5
126,523,V,Washington,33,39,30,31,133,240,241.5,-105,1.5
126,524,H,Atlanta,31,47,41,33,152,1.5,1,-115,119


In [213]:
display(sorted(bets['team'].unique()))

['ATL',
 'BOS',
 'BRK',
 'CHI',
 'CHO',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GSW',
 'HOU',
 'IND',
 'LAC',
 'LAL',
 'MEM',
 'MIA',
 'MIL',
 'MIN',
 'NOP',
 'NYK',
 'OKC',
 'ORL',
 'PHI',
 'PHO',
 'POR',
 'SAC',
 'SAS',
 'TOR',
 'UTA',
 'WAS']

In [214]:
team_abbreviations = {
    'Atlanta': 'ATL',
    'Boston': 'BOS',
    'Brooklyn': 'BRK',
    'Charlotte': 'CHO',
    'Chicago': 'CHI',
    'Cleveland': 'CLE',
    'Dallas': 'DAL',
    'Denver': 'DEN',
    'Detroit': 'DET',
    'GoldenState': 'GSW',
    'Houston': 'HOU',
    'Indiana': 'IND',
    'LAClippers': 'LAC',
    'LALakers': 'LAL',
    'Memphis': 'MEM',
    'Miami': 'MIA',
    'Milwaukee': 'MIL',
    'Minnesota': 'MIN',
    'NewOrleans': 'NOP',
    'NewYork': 'NYK',
    'OklahomaCity': 'OKC',
    'Orlando': 'ORL',
    'Philadelphia': 'PHI',
    'Phoenix': 'PHO',
    'Portland': 'POR',
    'Sacramento': 'SAC',
    'SanAntonio': 'SAS',
    'Toronto': 'TOR',
    'Utah': 'UTA',
    'Washington': 'WAS'
}

odds['Team'] = odds['Team'].map(team_abbreviations)
odds


,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
Date,,,,,,,,,,,,
1022,501,V,NOP,30,31,25,31,122,231.5,229.5,230,113
1022,502,H,TOR,27,29,32,29,130,6.5,6.5,-280,6
1022,503,V,LAL,25,29,31,17,102,227,3.5,-180,5
1022,504,H,LAC,22,40,23,27,112,1.5,224,150,110.5
1023,505,V,DET,27,27,29,36,119,210,211,240,104
...,...,...,...,...,...,...,...,...,...,...,...,...
126,522,H,NYK,30,25,30,25,110,221.5,2.5,-140,0.5
126,523,V,WAS,33,39,30,31,133,240,241.5,-105,1.5
126,524,H,ATL,31,47,41,33,152,1.5,1,-115,119


In [169]:
num_nan = odds['Team'].isnull().sum()
print(num_nan)


0


## Testing on this Seasons Data and Simulating the Bet Predictions
### We will assume a standard $100 bet per game, and will not include any forms of transaction costs
### We will determine the overall performance of the model the profit/loss from betting $100 on every game this season
### using our model thus far

In [170]:
bets = bets.sort_values("date")
bets = bets.reset_index(drop=True)
bets

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,37.0,85.0,0.435,13.0,33.0,0.394,15.0,21.0,...,22.2,42.4,203.0,116.0,LAC,112,1,2020,2019-10-22,False
1,240.0,240.0,42.0,81.0,0.519,11.0,31.0,0.355,17.0,24.0,...,25.0,35.8,300.0,125.0,LAL,102,0,2020,2019-10-22,True
2,265.0,265.0,42.0,103.0,0.408,14.0,40.0,0.350,32.0,38.0,...,25.0,37.7,146.0,124.0,NOP,122,0,2020,2019-10-22,True
3,265.0,265.0,43.0,102.0,0.422,19.0,45.0,0.422,17.0,20.0,...,22.2,35.7,158.0,114.0,TOR,130,1,2020,2019-10-22,False
4,240.0,240.0,33.0,88.0,0.375,9.0,34.0,0.265,10.0,15.0,...,34.0,32.2,140.0,95.0,ORL,94,1,2020,2019-10-23,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,240.0,240.0,48.0,93.0,0.516,13.0,28.0,0.464,20.0,21.0,...,42.9,33.4,250.0,134.0,POR,139,1,2020,2020-01-26,False
1380,240.0,240.0,44.0,84.0,0.524,8.0,19.0,0.421,18.0,26.0,...,100.0,31.7,126.0,111.0,PHO,109,0,2020,2020-01-26,True
1381,240.0,240.0,49.0,92.0,0.533,6.0,19.0,0.316,6.0,13.0,...,34.0,35.1,238.0,124.0,BRK,97,0,2020,2020-01-26,True
1382,240.0,240.0,40.0,88.0,0.455,11.0,33.0,0.333,19.0,27.0,...,22.2,29.2,167.0,117.0,DEN,117,1,2020,2020-01-26,False


In [171]:
del bets["mp.1"]
del bets["mp_opp.1"]
del bets["index_opp"]
bets

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,37.0,85.0,0.435,13.0,33.0,0.394,15.0,21.0,0.714,...,22.2,42.4,203.0,116.0,LAC,112,1,2020,2019-10-22,False
1,240.0,42.0,81.0,0.519,11.0,31.0,0.355,17.0,24.0,0.708,...,25.0,35.8,300.0,125.0,LAL,102,0,2020,2019-10-22,True
2,265.0,42.0,103.0,0.408,14.0,40.0,0.350,32.0,38.0,0.842,...,25.0,37.7,146.0,124.0,NOP,122,0,2020,2019-10-22,True
3,265.0,43.0,102.0,0.422,19.0,45.0,0.422,17.0,20.0,0.850,...,22.2,35.7,158.0,114.0,TOR,130,1,2020,2019-10-22,False
4,240.0,33.0,88.0,0.375,9.0,34.0,0.265,10.0,15.0,0.667,...,34.0,32.2,140.0,95.0,ORL,94,1,2020,2019-10-23,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,240.0,48.0,93.0,0.516,13.0,28.0,0.464,20.0,21.0,0.952,...,42.9,33.4,250.0,134.0,POR,139,1,2020,2020-01-26,False
1380,240.0,44.0,84.0,0.524,8.0,19.0,0.421,18.0,26.0,0.692,...,100.0,31.7,126.0,111.0,PHO,109,0,2020,2020-01-26,True
1381,240.0,49.0,92.0,0.533,6.0,19.0,0.316,6.0,13.0,0.462,...,34.0,35.1,238.0,124.0,BRK,97,0,2020,2020-01-26,True
1382,240.0,40.0,88.0,0.455,11.0,33.0,0.333,19.0,27.0,0.704,...,22.2,29.2,167.0,117.0,DEN,117,1,2020,2020-01-26,False


In [172]:
bets = bets.groupby("team", group_keys=False).apply(add_target)
bets["target"][pd.isnull(bets["target"])] = 2
bets["target"] = bets["target"].astype(int, errors="ignore")
bets

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,240.0,37.0,85.0,0.435,13.0,33.0,0.394,15.0,21.0,0.714,...,42.4,203.0,116.0,LAC,112,1,2020,2019-10-22,False,1
1,240.0,42.0,81.0,0.519,11.0,31.0,0.355,17.0,24.0,0.708,...,35.8,300.0,125.0,LAL,102,0,2020,2019-10-22,True,1
2,265.0,42.0,103.0,0.408,14.0,40.0,0.350,32.0,38.0,0.842,...,37.7,146.0,124.0,NOP,122,0,2020,2019-10-22,True,0
3,265.0,43.0,102.0,0.422,19.0,45.0,0.422,17.0,20.0,0.850,...,35.7,158.0,114.0,TOR,130,1,2020,2019-10-22,False,0
4,240.0,33.0,88.0,0.375,9.0,34.0,0.265,10.0,15.0,0.667,...,32.2,140.0,95.0,ORL,94,1,2020,2019-10-23,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,240.0,48.0,93.0,0.516,13.0,28.0,0.464,20.0,21.0,0.952,...,33.4,250.0,134.0,POR,139,1,2020,2020-01-26,False,2
1380,240.0,44.0,84.0,0.524,8.0,19.0,0.421,18.0,26.0,0.692,...,31.7,126.0,111.0,PHO,109,0,2020,2020-01-26,True,2
1381,240.0,49.0,92.0,0.533,6.0,19.0,0.316,6.0,13.0,0.462,...,35.1,238.0,124.0,BRK,97,0,2020,2020-01-26,True,2
1382,240.0,40.0,88.0,0.455,11.0,33.0,0.333,19.0,27.0,0.704,...,29.2,167.0,117.0,DEN,117,1,2020,2020-01-26,False,2


In [173]:
bets["won"].value_counts()
bets["target"].value_counts()

target
1    677
0    677
2     30
Name: count, dtype: int64

In [174]:
nulls = pd.isnull(bets).sum()
nulls = nulls[nulls > 0]
non_null_columns = bets.columns[~bets.columns.isin(nulls.index)]
bets = bets[non_null_columns].copy()
bets

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,240.0,37.0,85.0,0.435,13.0,33.0,0.394,15.0,21.0,0.714,...,42.4,203.0,116.0,LAC,112,1,2020,2019-10-22,False,1
1,240.0,42.0,81.0,0.519,11.0,31.0,0.355,17.0,24.0,0.708,...,35.8,300.0,125.0,LAL,102,0,2020,2019-10-22,True,1
2,265.0,42.0,103.0,0.408,14.0,40.0,0.350,32.0,38.0,0.842,...,37.7,146.0,124.0,NOP,122,0,2020,2019-10-22,True,0
3,265.0,43.0,102.0,0.422,19.0,45.0,0.422,17.0,20.0,0.850,...,35.7,158.0,114.0,TOR,130,1,2020,2019-10-22,False,0
4,240.0,33.0,88.0,0.375,9.0,34.0,0.265,10.0,15.0,0.667,...,32.2,140.0,95.0,ORL,94,1,2020,2019-10-23,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,240.0,48.0,93.0,0.516,13.0,28.0,0.464,20.0,21.0,0.952,...,33.4,250.0,134.0,POR,139,1,2020,2020-01-26,False,2
1380,240.0,44.0,84.0,0.524,8.0,19.0,0.421,18.0,26.0,0.692,...,31.7,126.0,111.0,PHO,109,0,2020,2020-01-26,True,2
1381,240.0,49.0,92.0,0.533,6.0,19.0,0.316,6.0,13.0,0.462,...,35.1,238.0,124.0,BRK,97,0,2020,2020-01-26,True,2
1382,240.0,40.0,88.0,0.455,11.0,33.0,0.333,19.0,27.0,0.704,...,29.2,167.0,117.0,DEN,117,1,2020,2020-01-26,False,2


In [175]:
non_stat_columns = ["season", "date", "won", "target", "team", "team_opp"]
stat_columns = bets.columns[~bets.columns.isin(non_stat_columns)]

bets[stat_columns] = scaler.fit_transform(bets[stat_columns])
bets

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,0.0,0.410256,0.415094,0.409877,0.454545,0.452381,0.511945,0.342857,0.380952,0.499124,...,0.247059,0.484043,0.4500,LAC,0.453488,1.0,2020,2019-10-22,False,1
1,0.0,0.538462,0.339623,0.617284,0.363636,0.404762,0.445392,0.400000,0.452381,0.488616,...,0.160784,1.000000,0.5625,LAL,0.337209,0.0,2020,2019-10-22,True,1
2,0.5,0.538462,0.754717,0.343210,0.500000,0.619048,0.436860,0.828571,0.785714,0.723292,...,0.185621,0.180851,0.5500,NOP,0.569767,0.0,2020,2019-10-22,True,0
3,0.5,0.564103,0.735849,0.377778,0.727273,0.738095,0.559727,0.400000,0.357143,0.737303,...,0.159477,0.244681,0.4250,TOR,0.662791,1.0,2020,2019-10-22,False,0
4,0.0,0.307692,0.471698,0.261728,0.272727,0.476190,0.291809,0.200000,0.238095,0.416813,...,0.113725,0.148936,0.1875,ORL,0.244186,1.0,2020,2019-10-23,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,0.0,0.692308,0.566038,0.609877,0.454545,0.333333,0.631399,0.485714,0.380952,0.915937,...,0.129412,0.734043,0.6750,POR,0.767442,1.0,2020,2020-01-26,False,2
1380,0.0,0.589744,0.396226,0.629630,0.227273,0.119048,0.558020,0.428571,0.500000,0.460595,...,0.107190,0.074468,0.3875,PHO,0.418605,0.0,2020,2020-01-26,True,2
1381,0.0,0.717949,0.547170,0.651852,0.136364,0.119048,0.378840,0.085714,0.190476,0.057793,...,0.151634,0.670213,0.5500,BRK,0.279070,0.0,2020,2020-01-26,True,2
1382,0.0,0.487179,0.471698,0.459259,0.363636,0.452381,0.407850,0.457143,0.523810,0.481611,...,0.074510,0.292553,0.4625,DEN,0.511628,1.0,2020,2020-01-26,False,2


In [176]:
bets.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

home
0.0    0.449422
1.0    0.550578
dtype: float64

In [177]:
bets_rolling = bets[list(stat_columns) + ["won", "team", "season"]]
bets_rolling = bets_rolling.groupby(["team", "season"], group_keys=False).apply(team_averages)
bets_rolling

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,won,team,season
0,0.0,0.410256,0.415094,0.409877,0.454545,0.452381,0.511945,0.342857,0.380952,0.499124,...,0.121622,0.158919,0.247059,0.484043,0.4500,0.453488,1.0,False,LAL,2020
1,0.0,0.538462,0.339623,0.617284,0.363636,0.404762,0.445392,0.400000,0.452381,0.488616,...,0.214286,0.189189,0.160784,1.000000,0.5625,0.337209,0.0,True,LAC,2020
2,0.5,0.538462,0.754717,0.343210,0.500000,0.619048,0.436860,0.828571,0.785714,0.723292,...,0.179537,0.189189,0.185621,0.180851,0.5500,0.569767,0.0,True,TOR,2020
3,0.5,0.564103,0.735849,0.377778,0.727273,0.738095,0.559727,0.400000,0.357143,0.737303,...,0.100386,0.158919,0.159477,0.244681,0.4250,0.662791,1.0,False,NOP,2020
4,0.0,0.307692,0.471698,0.261728,0.272727,0.476190,0.291809,0.200000,0.238095,0.416813,...,0.142857,0.286486,0.113725,0.148936,0.1875,0.244186,1.0,False,CLE,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,0.0,0.692308,0.566038,0.609877,0.454545,0.333333,0.631399,0.485714,0.380952,0.915937,...,0.239382,0.382703,0.129412,0.734043,0.6750,0.767442,1.0,False,IND,2020
1380,0.0,0.589744,0.396226,0.629630,0.227273,0.119048,0.558020,0.428571,0.500000,0.460595,...,0.173745,1.000000,0.107190,0.074468,0.3875,0.418605,0.0,True,MEM,2020
1381,0.0,0.717949,0.547170,0.651852,0.136364,0.119048,0.378840,0.085714,0.190476,0.057793,...,0.081081,0.286486,0.151634,0.670213,0.5500,0.279070,0.0,True,NYK,2020
1382,0.0,0.487179,0.471698,0.459259,0.363636,0.452381,0.407850,0.457143,0.523810,0.481611,...,0.057915,0.158919,0.074510,0.292553,0.4625,0.511628,1.0,False,HOU,2020


In [178]:
rolling_cols = [f"{col}_10" for col in bets_rolling.columns]
bets_rolling.columns = rolling_cols
bets = pd.concat([bets, bets_rolling], axis=1)

bets = bets.dropna()
bets = bets.reset_index(drop=True)

bets

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp_10,tov%_max_opp_10,usg%_max_opp_10,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,won_10,team_10,season_10
0,0.0,0.410256,0.415094,0.409877,0.454545,0.452381,0.511945,0.342857,0.380952,0.499124,...,0.121622,0.158919,0.247059,0.484043,0.4500,0.453488,1.0,False,LAL,2020
1,0.0,0.538462,0.339623,0.617284,0.363636,0.404762,0.445392,0.400000,0.452381,0.488616,...,0.214286,0.189189,0.160784,1.000000,0.5625,0.337209,0.0,True,LAC,2020
2,0.5,0.538462,0.754717,0.343210,0.500000,0.619048,0.436860,0.828571,0.785714,0.723292,...,0.179537,0.189189,0.185621,0.180851,0.5500,0.569767,0.0,True,TOR,2020
3,0.5,0.564103,0.735849,0.377778,0.727273,0.738095,0.559727,0.400000,0.357143,0.737303,...,0.100386,0.158919,0.159477,0.244681,0.4250,0.662791,1.0,False,NOP,2020
4,0.0,0.307692,0.471698,0.261728,0.272727,0.476190,0.291809,0.200000,0.238095,0.416813,...,0.142857,0.286486,0.113725,0.148936,0.1875,0.244186,1.0,False,CLE,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,0.0,0.692308,0.566038,0.609877,0.454545,0.333333,0.631399,0.485714,0.380952,0.915937,...,0.239382,0.382703,0.129412,0.734043,0.6750,0.767442,1.0,False,IND,2020
1380,0.0,0.589744,0.396226,0.629630,0.227273,0.119048,0.558020,0.428571,0.500000,0.460595,...,0.173745,1.000000,0.107190,0.074468,0.3875,0.418605,0.0,True,MEM,2020
1381,0.0,0.717949,0.547170,0.651852,0.136364,0.119048,0.378840,0.085714,0.190476,0.057793,...,0.081081,0.286486,0.151634,0.670213,0.5500,0.279070,0.0,True,NYK,2020
1382,0.0,0.487179,0.471698,0.459259,0.363636,0.452381,0.407850,0.457143,0.523810,0.481611,...,0.057915,0.158919,0.074510,0.292553,0.4625,0.511628,1.0,False,HOU,2020


In [179]:
bets["home_next"] = add_col(bets, "home")
bets["team_opp_next"] = add_col(bets, "team_opp")
bets["date_next"] = add_col(bets, "date")

bets

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,won_10,team_10,season_10,home_next,team_opp_next,date_next
0,0.0,0.410256,0.415094,0.409877,0.454545,0.452381,0.511945,0.342857,0.380952,0.499124,...,0.484043,0.4500,0.453488,1.0,False,LAL,2020,1.0,UTA,2019-10-25
1,0.0,0.538462,0.339623,0.617284,0.363636,0.404762,0.445392,0.400000,0.452381,0.488616,...,1.000000,0.5625,0.337209,0.0,True,LAC,2020,0.0,GSW,2019-10-24
2,0.5,0.538462,0.754717,0.343210,0.500000,0.619048,0.436860,0.828571,0.785714,0.723292,...,0.180851,0.5500,0.569767,0.0,True,TOR,2020,0.0,BOS,2019-10-25
3,0.5,0.564103,0.735849,0.377778,0.727273,0.738095,0.559727,0.400000,0.357143,0.737303,...,0.244681,0.4250,0.662791,1.0,False,NOP,2020,1.0,DAL,2019-10-25
4,0.0,0.307692,0.471698,0.261728,0.272727,0.476190,0.291809,0.200000,0.238095,0.416813,...,0.148936,0.1875,0.244186,1.0,False,CLE,2020,1.0,IND,2019-10-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,0.0,0.692308,0.566038,0.609877,0.454545,0.333333,0.631399,0.485714,0.380952,0.915937,...,0.734043,0.6750,0.767442,1.0,False,IND,2020,NaN,None,None
1380,0.0,0.589744,0.396226,0.629630,0.227273,0.119048,0.558020,0.428571,0.500000,0.460595,...,0.074468,0.3875,0.418605,0.0,True,MEM,2020,NaN,None,None
1381,0.0,0.717949,0.547170,0.651852,0.136364,0.119048,0.378840,0.085714,0.190476,0.057793,...,0.670213,0.5500,0.279070,0.0,True,NYK,2020,NaN,None,None
1382,0.0,0.487179,0.471698,0.459259,0.363636,0.452381,0.407850,0.457143,0.523810,0.481611,...,0.292553,0.4625,0.511628,1.0,False,HOU,2020,NaN,None,None


In [180]:
bet_matchup = bets.merge(bets[rolling_cols + ["team_opp_next", "date_next", "team"]], 
                          left_on=["team", "date_next"], right_on=["team_opp_next", "date_next"])

bet_matchup

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp_10_y,ortg_max_opp_10_y,drtg_max_opp_10_y,total_opp_10_y,home_opp_10_y,won_10_y,team_10_y,season_10_y,team_opp_next_y,team_y
0,0.0,0.410256,0.415094,0.409877,0.454545,0.452381,0.511945,0.342857,0.380952,0.499124,...,0.107190,0.468085,0.3500,0.255814,0.0,True,UTA,2020,LAL,UTA
1,0.5,0.538462,0.754717,0.343210,0.500000,0.619048,0.436860,0.828571,0.785714,0.723292,...,0.100654,0.734043,0.2625,0.395349,1.0,False,BOS,2020,TOR,BOS
2,0.5,0.564103,0.735849,0.377778,0.727273,0.738095,0.559727,0.400000,0.357143,0.737303,...,0.189542,0.627660,0.3750,0.313953,0.0,True,DAL,2020,NOP,DAL
3,0.0,0.307692,0.471698,0.261728,0.272727,0.476190,0.291809,0.200000,0.238095,0.416813,...,0.224837,0.271277,0.4625,0.534884,0.0,False,IND,2020,CLE,IND
4,0.0,0.358974,0.245283,0.474074,0.363636,0.523810,0.361775,0.685714,0.666667,0.681261,...,0.159477,0.244681,0.4250,0.662791,1.0,False,NOP,2020,DAL,NOP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1347,0.0,0.666667,0.528302,0.609877,0.363636,0.428571,0.426621,0.657143,0.619048,0.718039,...,0.179085,0.494681,0.6500,0.383721,1.0,True,DEN,2020,HOU,DEN
1348,0.0,0.487179,0.433962,0.483951,0.500000,0.500000,0.522184,0.342857,0.309524,0.707531,...,0.152941,0.377660,0.5000,0.465116,0.0,False,NOP,2020,BOS,NOP
1349,0.0,0.384615,0.509434,0.323457,0.409091,0.547619,0.392491,0.314286,0.333333,0.539405,...,0.132026,0.287234,0.6125,0.511628,1.0,True,LAC,2020,ORL,LAC
1350,0.0,0.435897,0.358491,0.479012,0.409091,0.309524,0.597270,0.428571,0.595238,0.299475,...,0.053595,0.941489,0.5000,0.290698,1.0,True,BOS,2020,NOP,BOS


In [196]:
bet_matchup['date_next']

0       2019-10-25
1       2019-10-25
2       2019-10-25
3       2019-10-26
4       2019-10-25
           ...    
1347    2020-01-26
1348    2020-01-26
1349    2020-01-26
1350    2020-01-26
1351    2020-01-26
Name: date_next, Length: 1352, dtype: object

In [197]:
bet_matchup = bet_matchup.sort_values("date_next")
bet_matchup


,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp_10_y,ortg_max_opp_10_y,drtg_max_opp_10_y,total_opp_10_y,home_opp_10_y,won_10_y,team_10_y,season_10_y,team_opp_next_y,team_y
0,0.0,0.410256,0.415094,0.409877,0.454545,0.452381,0.511945,0.342857,0.380952,0.499124,...,0.107190,0.468085,0.3500,0.255814,0.0,True,UTA,2020,LAL,UTA
18,0.5,0.641026,0.849057,0.387654,0.454545,0.690476,0.354949,0.542857,0.547619,0.625219,...,0.138562,0.526596,0.6375,0.604651,0.0,True,CHO,2020,MIN,CHO
17,0.0,0.435897,0.490566,0.390123,0.090909,0.428571,0.105802,0.485714,0.595238,0.416813,...,0.118954,0.510638,0.6000,0.616279,1.0,False,CHI,2020,MEM,CHI
22,0.0,0.717949,0.660377,0.570370,0.363636,0.452381,0.407850,0.342857,0.309524,0.707531,...,0.095425,0.095745,0.4875,0.313953,1.0,True,DEN,2020,PHO,DEN
15,0.0,0.717949,0.792453,0.488889,0.272727,0.380952,0.351536,0.428571,0.404762,0.681261,...,0.129412,0.143617,0.2375,0.546512,1.0,False,MEM,2020,CHI,MEM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336,0.0,0.487179,0.811321,0.266667,0.454545,0.642857,0.380546,0.485714,0.357143,1.000000,...,0.220915,0.191489,0.6000,0.593023,1.0,True,HOU,2020,DEN,HOU
1335,0.0,0.743590,0.452830,0.755556,0.636364,0.333333,0.875427,0.257143,0.190476,0.865149,...,0.189542,0.292553,0.7125,0.697674,0.0,False,POR,2020,IND,POR
1334,0.0,0.461538,0.396226,0.481481,0.227273,0.357143,0.310580,0.400000,0.380952,0.667250,...,0.121569,0.452128,0.6750,0.453488,1.0,True,MEM,2020,PHO,MEM
1343,0.0,0.307692,0.283019,0.380247,0.090909,0.261905,0.180887,0.714286,0.666667,0.733800,...,0.052288,0.515957,0.6750,0.453488,1.0,True,TOR,2020,SAS,TOR


In [198]:
bet_matchup[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]]

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
0,LAL,UTA,UTA,LAL,2019-10-25
18,MIN,CHO,CHO,MIN,2019-10-25
17,MEM,CHI,CHI,MEM,2019-10-25
22,PHO,DEN,DEN,PHO,2019-10-25
15,CHI,MEM,MEM,CHI,2019-10-25
...,...,...,...,...,...
1336,DEN,HOU,HOU,DEN,2020-01-26
1335,IND,POR,POR,IND,2020-01-26
1334,PHO,MEM,MEM,PHO,2020-01-26
1343,SAS,TOR,TOR,SAS,2020-01-26


In [202]:
bet_matchup[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]][bet_matchup['date_next'] == '2019-10-25']


,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
0,LAL,UTA,UTA,LAL,2019-10-25
18,MIN,CHO,CHO,MIN,2019-10-25
17,MEM,CHI,CHI,MEM,2019-10-25
22,PHO,DEN,DEN,PHO,2019-10-25
15,CHI,MEM,MEM,CHI,2019-10-25
14,CHO,MIN,MIN,CHO,2019-10-25
12,BOS,TOR,TOR,BOS,2019-10-25
11,SAC,POR,POR,SAC,2019-10-25
21,DEN,PHO,PHO,DEN,2019-10-25
10,UTA,LAL,LAL,UTA,2019-10-25


In [203]:
non_stat_columns = list(bet_matchup.columns[bet_matchup.dtypes == "object"]) + non_stat_columns

non_stat_columns

['team_x',
 'team_opp',
 'date',
 'team_10_x',
 'team_opp_next_x',
 'date_next',
 'team_10_y',
 'team_opp_next_y',
 'team_y',
 'team_x',
 'team_opp',
 'date',
 'team_10_x',
 'team_opp_next_x',
 'date_next',
 'team_10_y',
 'team_opp_next_y',
 'team_y',
 'season',
 'date',
 'won',
 'target',
 'team',
 'team_opp']

In [204]:
# X_columns_bet = bet_matchup.columns[~bet_matchup.columns.isin(non_stat_columns)]
# X_bet = bet_matchup[X_columns_bet]
# y_bet = bet_matchup["target"]
# display(X_bet)
# display(y_bet)

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,stl%_max_opp_10_y,blk%_max_opp_10_y,tov%_max_opp_10_y,usg%_max_opp_10_y,ortg_max_opp_10_y,drtg_max_opp_10_y,total_opp_10_y,home_opp_10_y,won_10_y,season_10_y
0,0.0,0.410256,0.415094,0.409877,0.454545,0.452381,0.511945,0.342857,0.380952,0.499124,...,0.048387,0.299228,0.494054,0.107190,0.468085,0.3500,0.255814,0.0,True,2020
18,0.5,0.641026,0.849057,0.387654,0.454545,0.690476,0.354949,0.542857,0.547619,0.625219,...,0.182028,0.086873,0.189189,0.138562,0.526596,0.6375,0.604651,0.0,True,2020
17,0.0,0.435897,0.490566,0.390123,0.090909,0.428571,0.105802,0.485714,0.595238,0.416813,...,0.066820,0.117761,1.000000,0.118954,0.510638,0.6000,0.616279,1.0,False,2020
22,0.0,0.717949,0.660377,0.570370,0.363636,0.452381,0.407850,0.342857,0.309524,0.707531,...,0.225806,0.144788,0.459459,0.095425,0.095745,0.4875,0.313953,1.0,True,2020
15,0.0,0.717949,0.792453,0.488889,0.272727,0.380952,0.351536,0.428571,0.404762,0.681261,...,0.073733,0.428571,1.000000,0.129412,0.143617,0.2375,0.546512,1.0,False,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336,0.0,0.487179,0.811321,0.266667,0.454545,0.642857,0.380546,0.485714,0.357143,1.000000,...,0.096774,0.254826,0.405405,0.220915,0.191489,0.6000,0.593023,1.0,True,2020
1335,0.0,0.743590,0.452830,0.755556,0.636364,0.333333,0.875427,0.257143,0.190476,0.865149,...,0.066820,0.235521,0.063784,0.189542,0.292553,0.7125,0.697674,0.0,False,2020
1334,0.0,0.461538,0.396226,0.481481,0.227273,0.357143,0.310580,0.400000,0.380952,0.667250,...,0.096774,0.057915,0.459459,0.121569,0.452128,0.6750,0.453488,1.0,True,2020
1343,0.0,0.307692,0.283019,0.380247,0.090909,0.261905,0.180887,0.714286,0.666667,0.733800,...,0.105991,0.177606,0.278919,0.052288,0.515957,0.6750,0.453488,1.0,True,2020


0       1
18      1
17      0
22      0
15      1
       ..
1336    1
1335    0
1334    0
1343    0
1351    0
Name: target, Length: 1352, dtype: int64

In [205]:
# y_bet.value_counts()

target
1    676
0    676
Name: count, dtype: int64

In [206]:
# X_bet = X_bet.astype('float32')
# y_bet = y_bet.astype('float32')

In [207]:
# y_bet_pred = rnn_model.predict(X_bet)

43/43 [==============================] - 5s 105ms/step


In [208]:
# y_bet_pred_classes = np.argmax(y_bet_pred, axis=1)
# bet_accuracy = accuracy_score(y_bet, y_bet_pred_classes)
# print("Accuracy:", bet_accuracy)

# print(classification_report(y_bet, y_bet_pred_classes))

Accuracy: 0.5
              precision    recall  f1-score   support

         0.0       0.50      1.00      0.67       676
         1.0       0.00      0.00      0.00       676

    accuracy                           0.50      1352
   macro avg       0.25      0.50      0.33      1352
weighted avg       0.25      0.50      0.33      1352



In [215]:
odds

,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
Date,,,,,,,,,,,,
1022,501,V,NOP,30,31,25,31,122,231.5,229.5,230,113
1022,502,H,TOR,27,29,32,29,130,6.5,6.5,-280,6
1022,503,V,LAL,25,29,31,17,102,227,3.5,-180,5
1022,504,H,LAC,22,40,23,27,112,1.5,224,150,110.5
1023,505,V,DET,27,27,29,36,119,210,211,240,104
...,...,...,...,...,...,...,...,...,...,...,...,...
126,522,H,NYK,30,25,30,25,110,221.5,2.5,-140,0.5
126,523,V,WAS,33,39,30,31,133,240,241.5,-105,1.5
126,524,H,ATL,31,47,41,33,152,1.5,1,-115,119


In [216]:
odds.reset_index(inplace=True)
odds['date'] = odds.index
odds


,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,date
0,1022,501,V,NOP,30,31,25,31,122,231.5,229.5,230,113,0
1,1022,502,H,TOR,27,29,32,29,130,6.5,6.5,-280,6,1
2,1022,503,V,LAL,25,29,31,17,102,227,3.5,-180,5,2
3,1022,504,H,LAC,22,40,23,27,112,1.5,224,150,110.5,3
4,1023,505,V,DET,27,27,29,36,119,210,211,240,104,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,126,522,H,NYK,30,25,30,25,110,221.5,2.5,-140,0.5,1379
1380,126,523,V,WAS,33,39,30,31,133,240,241.5,-105,1.5,1380
1381,126,524,H,ATL,31,47,41,33,152,1.5,1,-115,119,1381
1382,126,525,V,IND,27,29,37,36,129,220.5,221,130,1.5,1382


In [232]:
odds_test = odds.copy()
odds_test

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,date
0,1022,501,V,NOP,30,31,25,31,122,231.5,229.5,230,113,0
1,1022,502,H,TOR,27,29,32,29,130,6.5,6.5,-280,6,1
2,1022,503,V,LAL,25,29,31,17,102,227,3.5,-180,5,2
3,1022,504,H,LAC,22,40,23,27,112,1.5,224,150,110.5,3
4,1023,505,V,DET,27,27,29,36,119,210,211,240,104,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,126,522,H,NYK,30,25,30,25,110,221.5,2.5,-140,0.5,1379
1380,126,523,V,WAS,33,39,30,31,133,240,241.5,-105,1.5,1380
1381,126,524,H,ATL,31,47,41,33,152,1.5,1,-115,119,1381
1382,126,525,V,IND,27,29,37,36,129,220.5,221,130,1.5,1382


In [233]:
# Convert all values in date to strings and add a '-' between every two characters
odds_test['Date'] = odds_test['Date'].astype(str).apply(lambda x: '-'.join(x[i:i+2] for i in range(0, len(x), 2)))
odds_test['Date']


0       10-22
1       10-22
2       10-22
3       10-22
4       10-23
        ...  
1379     12-6
1380     12-6
1381     12-6
1382     12-6
1383     12-6
Name: Date, Length: 1384, dtype: object

In [234]:
odds_test['Date'] = odds_test['Date'].apply(lambda x: '2019-' + x if x[:2] in ['10', '11', '12'] else '2020-' + x)
odds_test

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,date
0,2019-10-22,501,V,NOP,30,31,25,31,122,231.5,229.5,230,113,0
1,2019-10-22,502,H,TOR,27,29,32,29,130,6.5,6.5,-280,6,1
2,2019-10-22,503,V,LAL,25,29,31,17,102,227,3.5,-180,5,2
3,2019-10-22,504,H,LAC,22,40,23,27,112,1.5,224,150,110.5,3
4,2019-10-23,505,V,DET,27,27,29,36,119,210,211,240,104,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,2019-12-6,522,H,NYK,30,25,30,25,110,221.5,2.5,-140,0.5,1379
1380,2019-12-6,523,V,WAS,33,39,30,31,133,240,241.5,-105,1.5,1380
1381,2019-12-6,524,H,ATL,31,47,41,33,152,1.5,1,-115,119,1381
1382,2019-12-6,525,V,IND,27,29,37,36,129,220.5,221,130,1.5,1382


In [235]:
odds_test = odds_test.drop('date', axis=1)
odds_test

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,2019-10-22,501,V,NOP,30,31,25,31,122,231.5,229.5,230,113
1,2019-10-22,502,H,TOR,27,29,32,29,130,6.5,6.5,-280,6
2,2019-10-22,503,V,LAL,25,29,31,17,102,227,3.5,-180,5
3,2019-10-22,504,H,LAC,22,40,23,27,112,1.5,224,150,110.5
4,2019-10-23,505,V,DET,27,27,29,36,119,210,211,240,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,2019-12-6,522,H,NYK,30,25,30,25,110,221.5,2.5,-140,0.5
1380,2019-12-6,523,V,WAS,33,39,30,31,133,240,241.5,-105,1.5
1381,2019-12-6,524,H,ATL,31,47,41,33,152,1.5,1,-115,119
1382,2019-12-6,525,V,IND,27,29,37,36,129,220.5,221,130,1.5


In [239]:
merged_df = bet_matchup.merge(odds_test, left_on=['team_x', 'date_next'], right_on=['Team', 'Date'])
print(merged_df.shape)
merged_df

(974, 438)


,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,0.0,0.410256,0.415094,0.409877,0.454545,0.452381,0.511945,0.342857,0.380952,0.499124,...,LAL,24,19,31,21,95,2,3.5,-160,pk
1,0.5,0.641026,0.849057,0.387654,0.454545,0.690476,0.354949,0.542857,0.547619,0.625219,...,MIN,30,28,45,18,121,4,5,-220,2.5
2,0.0,0.435897,0.490566,0.390123,0.090909,0.428571,0.105802,0.485714,0.595238,0.416813,...,MEM,29,31,21,21,102,1,224,105,116
3,0.0,0.717949,0.660377,0.570370,0.363636,0.452381,0.407850,0.342857,0.309524,0.707531,...,PHO,21,24,17,33,107,221,219.5,650,111.5
4,0.0,0.717949,0.792453,0.488889,0.272727,0.380952,0.351536,0.428571,0.404762,0.681261,...,CHI,25,22,32,31,110,220.5,2,-125,5.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,0.0,0.615385,0.566038,0.530864,0.681818,0.595238,0.627986,0.285714,0.285714,0.588441,...,GSW,32,23,20,25,113,220,217,525,106.5
970,0.0,0.256410,0.283019,0.316049,0.181818,0.428571,0.213311,0.714286,0.738095,0.611208,...,BOS,24,26,30,29,109,6.5,7.5,-400,2.5
971,0.0,0.307692,0.415094,0.293827,0.136364,0.333333,0.204778,0.657143,0.571429,0.819615,...,IND,29,38,29,19,115,211.5,3.5,-155,108
972,0.5,0.538462,0.490566,0.501235,0.318182,0.380952,0.407850,0.542857,0.571429,0.577933,...,PHI,27,16,23,31,97,1,212.5,135,4.5


In [240]:
filtered_df = merged_df.drop(columns=odds_test.columns)
print(filtered_df.shape)
filtered_df


(974, 425)


,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp_10_y,ortg_max_opp_10_y,drtg_max_opp_10_y,total_opp_10_y,home_opp_10_y,won_10_y,team_10_y,season_10_y,team_opp_next_y,team_y
0,0.0,0.410256,0.415094,0.409877,0.454545,0.452381,0.511945,0.342857,0.380952,0.499124,...,0.107190,0.468085,0.3500,0.255814,0.0,True,UTA,2020,LAL,UTA
1,0.5,0.641026,0.849057,0.387654,0.454545,0.690476,0.354949,0.542857,0.547619,0.625219,...,0.138562,0.526596,0.6375,0.604651,0.0,True,CHO,2020,MIN,CHO
2,0.0,0.435897,0.490566,0.390123,0.090909,0.428571,0.105802,0.485714,0.595238,0.416813,...,0.118954,0.510638,0.6000,0.616279,1.0,False,CHI,2020,MEM,CHI
3,0.0,0.717949,0.660377,0.570370,0.363636,0.452381,0.407850,0.342857,0.309524,0.707531,...,0.095425,0.095745,0.4875,0.313953,1.0,True,DEN,2020,PHO,DEN
4,0.0,0.717949,0.792453,0.488889,0.272727,0.380952,0.351536,0.428571,0.404762,0.681261,...,0.129412,0.143617,0.2375,0.546512,1.0,False,MEM,2020,CHI,MEM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,0.0,0.615385,0.566038,0.530864,0.681818,0.595238,0.627986,0.285714,0.285714,0.588441,...,0.397386,0.276596,0.7625,0.418605,0.0,True,SAS,2020,GSW,SAS
970,0.0,0.256410,0.283019,0.316049,0.181818,0.428571,0.213311,0.714286,0.738095,0.611208,...,0.066667,0.212766,0.3875,0.511628,1.0,False,CHO,2020,BOS,CHO
971,0.0,0.307692,0.415094,0.293827,0.136364,0.333333,0.204778,0.657143,0.571429,0.819615,...,0.087582,0.388298,0.5125,0.511628,1.0,False,PHI,2020,IND,PHI
972,0.5,0.538462,0.490566,0.501235,0.318182,0.380952,0.407850,0.542857,0.571429,0.577933,...,0.461438,0.734043,0.3375,0.546512,1.0,False,IND,2020,PHI,IND


In [241]:
X_columns_bet = filtered_df.columns[~filtered_df.columns.isin(non_stat_columns)]
X_bet = filtered_df[X_columns_bet]
y_bet = filtered_df["target"]
display(X_bet)
display(y_bet)

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,stl%_max_opp_10_y,blk%_max_opp_10_y,tov%_max_opp_10_y,usg%_max_opp_10_y,ortg_max_opp_10_y,drtg_max_opp_10_y,total_opp_10_y,home_opp_10_y,won_10_y,season_10_y
0,0.0,0.410256,0.415094,0.409877,0.454545,0.452381,0.511945,0.342857,0.380952,0.499124,...,0.048387,0.299228,0.494054,0.107190,0.468085,0.3500,0.255814,0.0,True,2020
1,0.5,0.641026,0.849057,0.387654,0.454545,0.690476,0.354949,0.542857,0.547619,0.625219,...,0.182028,0.086873,0.189189,0.138562,0.526596,0.6375,0.604651,0.0,True,2020
2,0.0,0.435897,0.490566,0.390123,0.090909,0.428571,0.105802,0.485714,0.595238,0.416813,...,0.066820,0.117761,1.000000,0.118954,0.510638,0.6000,0.616279,1.0,False,2020
3,0.0,0.717949,0.660377,0.570370,0.363636,0.452381,0.407850,0.342857,0.309524,0.707531,...,0.225806,0.144788,0.459459,0.095425,0.095745,0.4875,0.313953,1.0,True,2020
4,0.0,0.717949,0.792453,0.488889,0.272727,0.380952,0.351536,0.428571,0.404762,0.681261,...,0.073733,0.428571,1.000000,0.129412,0.143617,0.2375,0.546512,1.0,False,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,0.0,0.615385,0.566038,0.530864,0.681818,0.595238,0.627986,0.285714,0.285714,0.588441,...,0.158986,0.169884,0.278919,0.397386,0.276596,0.7625,0.418605,0.0,True,2020
970,0.0,0.256410,0.283019,0.316049,0.181818,0.428571,0.213311,0.714286,0.738095,0.611208,...,0.112903,0.200772,0.192432,0.066667,0.212766,0.3875,0.511628,1.0,False,2020
971,0.0,0.307692,0.415094,0.293827,0.136364,0.333333,0.204778,0.657143,0.571429,0.819615,...,0.057604,0.057915,0.154595,0.087582,0.388298,0.5125,0.511628,1.0,False,2020
972,0.5,0.538462,0.490566,0.501235,0.318182,0.380952,0.407850,0.542857,0.571429,0.577933,...,0.110599,0.077220,0.137297,0.461438,0.734043,0.3375,0.546512,1.0,False,2020


0      1
1      1
2      0
3      0
4      1
      ..
969    0
970    1
971    1
972    0
973    0
Name: target, Length: 974, dtype: int64